In [26]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from transformers import BertModel

# SENT_MODEL = 'skt/kobert-base-v1'
SENT_MODEL = '/kobert-nsmc-sentiment'

In [11]:
from kobert_tokenizer import KoBERTTokenizer

# 일반적인 토크나이저 사용시
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained(SENT_MODEL)
tokenizer.encode("한국어 모델을 공유합니다.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]

In [14]:
# KoBERT 모델을 라이브러리로 로드

import torch
from transformers import BertModel
model = BertModel.from_pretrained(SENT_MODEL)
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
            attention_mask = torch.tensor(inputs['attention_mask']))

out.pooler_output.shape

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


torch.Size([1, 768])

In [ ]:
# KoBERT 모델을 라이브러리로 로드

import torch
from transformers import BertModel, AutoTokenizer, AutoModelForSequenceClassification

LABEL_MAP = {0:"부정", 1:"긍정"}
SENT_MODEL = 'monologg/koelectra-small-v2-discriminator'
tokenizer = AutoTokenizer.from_pretrained(SENT_MODEL, use_fast=False)

model =  AutoModelForSequenceClassification.from_pretrained(SENT_MODEL)
model.eval()
# text = "한국어 모델을 공유합니다."
text = "음식이 다 식어서 맛이 없었어요"

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)[0].tolist()
    pred = int(torch.argmax(logits, dim=-1).item())

print(
    f'label_id : {pred} \n'
    f'label : {LABEL_MAP.get(pred, "unknown")} \n'
    f'confidence: {float(max(probs))} \n'
    f'probs : {probs}'
    )

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


label_id : 1 
label : 긍정 
confidence: 0.5081313252449036 
probs : [0.49186864495277405, 0.5081313252449036]
